# **INGENIERÍA LINGÜÍSTICA. PRÁCTICA 2**

En este cuaderno se encuentra toda la implementación de la práctica 2 de la asignatura de Ingeniería Lingüística del máster de Inteligencia Artifical de la UPM.

El cuaderno se va a dividir en las siguientes partes:
* 1. Imports
* 2. Funciones de carga y procesado de documentos
* 3. Implementación del algoritmo de clasificación
* 4. Pruebas y evaluación

## 1. Imports

Se han utilizado diversas librerías de python:

In [1]:
import glob
import io
import json
import numpy as np
import math
import re
import random
from sklearn import preprocessing  # to normalise existing X
from collections import Counter 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric
from prettytable import PrettyTable

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.data import load
from string import punctuation

from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alexb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 2. Funciones de carga y procesado de documentos

En este apartado se detallan las funciones utilizadas para el preprocesado de los textos y la extracción de vocabulario. 

### 2.1. Parámetros

Estos parámetros se utilizan en distintas funciones del código. Se ajustan para probar la mejora o empeoramiento de nuestro modelo. Estos parámetros son:

* **Número mínimo de aparición**: son tres parámetros distintos que se usan para determinar cuál es el número mínimo de apariciones que debe tener una palabra en un conjunto de textos de un mismo tema para ser considerada importante o de interés para la clasificación
    * **min_deportes**: número mínimo de veces que debe aparecer una palabra en los textos de deportes para que sea considerada
    * **min_politica**: número mínimo de veces que debe aparecer una palabra en los textos de política para que sea considerada
    * **min_salud**: número mínimo de veces que debe aparecer una palabra en los textos de salud para que sea considerada

In [3]:
#Parámetros usados
min_deportes = 9
min_politica = 10
min_salud = 9

### 2.2. Funciones de procesado de documentos y obtención del glosario

A continuación, se encuentran las funciones que se han utilizado para obtener el glosario de términos.

In [4]:
#Función de carga de los documentos
def load_documents(filenames):
    return [io.open(name, 'r', encoding='utf-8').read().replace("\r", "").replace("\n", "") for name in filenames]

In [5]:
#Función que comprueba la etiqueta original de cada documento
#Usada para comprobar la bondad de nuestro clasificador y expresar el porcentaje de aciertos de cada documento
def check_original_labels(filenames):
    labels = []
    
    #Se aprovecha que cada documento tiene en su nombre la etiqueta a la que pertenece originalmente
    for name in filenames:
        if 'deportes' in name:
            labels.append(0)
        elif 'politica' in name:
            labels.append(1)
        elif 'salud' in name:
            labels.append(2)
        else:
            labels.append(random.randint(0,3))
            
    #Devolución de la lista de etiquetas reales
    return labels

In [6]:
#Funcion tokenize (extracción de términos relevantes)
def tokenize(text):
    
    #Lista de palabras vacías (stopwords): combinación entre archivo .json + lista de palabras extra + stopwords en español e inglés del paquete stopwords
    add_stopwords = ['tan', 'través', 'aún', 'sun', 'ser', 'estar', 'tener', 'haber', 'efe', 'además', 'aun']
    stopwords_json = json.load(open('stop_words_spanish.json',"rb"), encoding="utf-8")
    stopwords_spanish = stopwords.words('spanish') + add_stopwords + stopwords.words('english') + stopwords_json
    
    #Lista de signos de puntuación
    non_words = list(punctuation)
    non_words.extend(['¿', '¡','”', '“','«','º','ºc','»','‘','’','…','–','—','—','ª'])
    non_words.extend(map(str,range(10)))
    
    #Obtención de los términos de los textos sin los signos de puntuación y demás elementos que no son palabras
    text = ''.join([c.lower() for c in text if c not in non_words])
    
    #Eliminación de las stopwords de la lista anterior y devolución de la lista final de términos
    tokens_with_stopwords = re.findall("[A-Za-zÀ-ÖØ-öø-ÿ]{3,}", text)
    tokens = [token for token in tokens_with_stopwords if token not in stopwords_spanish]
    return tokens

In [7]:
#Funcion usada para contar el número de apariciones de las palabras en un texto
def count_words(documents):
    
    #Lista de palabras encontradas en los documentos tras pasar por el tokenizer
    wordstring = ''
    for document in documents:
        wordstring += " ".join(str(item) for item in tokenize(document))
    
    wordlist = wordstring.split()
    
    #Lista de frecuencias de las palabras
    wordfreq = []
    for w in wordlist:
        wordfreq.append(wordlist.count(w))

    return wordlist, wordfreq

### 2.3. Glosario de deportes

In [8]:
#Carga de los documentos pertenecientes al apartado de deportes
deportes = load_documents(sorted(glob.glob('Deportes/*')))

In [9]:
#Se seleccionan los documentos de test para hacer el glosario sólo con ellos
wordlist, wordfreq = count_words(deportes)

#Se forma la lista de términos
deportes_new_vocab = []
for word, count in zip(wordlist,wordfreq):
    
    #Se comprueba que los términos aparezcan un mínimo de veces y no estén ya incluídos previamente en el glosario
    if(count >= min_deportes and word not in deportes_new_vocab):
        
        #La lista final será un glosario de cada tema sin palabras repetidas
        deportes_new_vocab.append(word)

variab_deportes = ['actividades', 'competiciones', 'conjuntos', 'cuartos', 'deportes', 'deportiva', 'deportivas', 
                   'deportivos', 'encuentros', 'española', 'españolas', 'españoles', 'finales', 'frentes', 'grupos', 
                   'historias', 'hora', 'jugador', 'jugadora', 'jugadoras', 'ligas', 'minuto', 'partidos', 'presidentes', 
                   'pruebas', 'reales', 'rivales']
deportes_new_vocab = deportes_new_vocab + variab_deportes

deportes_new_vocab.sort()
print(deportes_new_vocab)

['actividad', 'actividades', 'anna', 'atlético', 'años', 'baloncesto', 'china', 'competiciones', 'competición', 'conjunto', 'conjuntos', 'cuarto', 'cuartos', 'deporte', 'deportes', 'deportiva', 'deportivas', 'deportivo', 'deportivos', 'diego', 'encuentro', 'encuentros', 'equipo', 'equipos', 'españa', 'español', 'española', 'españolas', 'españoles', 'final', 'finales', 'frente', 'frentes', 'grupo', 'grupos', 'historia', 'historias', 'hora', 'horas', 'juan', 'jugador', 'jugadora', 'jugadoras', 'jugadores', 'liga', 'ligas', 'madrid', 'maradona', 'minuto', 'minutos', 'partido', 'partidos', 'popular', 'presidente', 'presidentes', 'prueba', 'pruebas', 'real', 'reales', 'rival', 'rivales', 'samaranch']


### 2.4. Glosario de política

In [10]:
#Carga de los documentos pertenecientes al apartado de política
politica = load_documents(sorted(glob.glob('Política/*')))

In [11]:
#Se seleccionan los documentos de test para hacer el glosario sólo con ellos
wordlist, wordfreq = count_words(politica)

#Se forma la lista de términos
politica_new_vocab = []

for word, count in zip(wordlist,wordfreq):
    
    #Se comprueba que los términos aparezcan un mínimo de veces y no estén ya incluídos previamente en el glosario
    if(count >= min_politica and word not in politica_new_vocab):
        
        #La lista final será un glosario de cada tema sin palabras repetidas
        politica_new_vocab.append(word)


variab_politica = ['acuerdos', 'audiencias', 'británicos', 'británicas', 'británica', 'casos', 'comisiones', 'congresos', 
                   'consejos', 'día', 'empresa', 'europeas', 'europeos', 'formaciones', 'función', 'generales', 
                   'gobiernos', 'judiciales', 'jueces', 'jueza', 'juezas', 'leyes', 'mercados', 'meses', 'ministros', 
                   'ministra', 'ministras', 'nacionales', 'oposiciones', 'pactos', 'partidos', 'países', 'policías', 
                   'problemas', 'reina', 'reyes', 'reinas', 'saudíes', 'tribunales']
politica_new_vocab = politica_new_vocab + variab_politica

politica_new_vocab.sort()
print(politica_new_vocab)

['acuerdo', 'acuerdos', 'audiencia', 'audiencias', 'año', 'años', 'brexit', 'británica', 'británicas', 'británico', 'británicos', 'bruselas', 'carlos', 'casado', 'caso', 'casos', 'cgpj', 'comisiones', 'comisión', 'congreso', 'congresos', 'consejo', 'consejos', 'crisis', 'día', 'días', 'empresa', 'empresas', 'enero', 'españa', 'europea', 'europeas', 'europeo', 'europeos', 'fernández', 'formaciones', 'formación', 'funciones', 'función', 'general', 'generales', 'gobierno', 'gobiernos', 'interior', 'israel', 'johnson', 'judicial', 'judiciales', 'jueces', 'juez', 'jueza', 'juezas', 'ley', 'leyes', 'londres', 'lunes', 'madrid', 'mercado', 'mercados', 'mes', 'meses', 'ministra', 'ministras', 'ministro', 'ministros', 'moncloa', 'nacional', 'nacionales', 'oposiciones', 'oposición', 'pablo', 'pacto', 'pactos', 'partido', 'partidos', 'país', 'países', 'policía', 'policías', 'política', 'presidente', 'problema', 'problemas', 'psoe', 'reina', 'reinas', 'reino', 'renovar', 'rey', 'reyes', 'saudí', '

### 2.5. Glosario de salud

In [12]:
#Carga de los documentos pertenecientes al apartado de salud
salud = load_documents(sorted(glob.glob('Salud/*')))

In [13]:
#Se seleccionan los documentos de test para hacer el glosario sólo con ellos
wordlist, wordfreq = count_words(salud)

#Se forma la lista de términos
salud_new_vocab = []
for word, count in zip(wordlist,wordfreq):
    
    #Se comprueba que los términos aparezcan un mínimo de veces y no estén ya incluídos previamente en el glosario
    if(count >= min_salud and word not in salud_new_vocab):
        
        #La lista final será un glosario de cada tema sin palabras repetidas
        salud_new_vocab.append(word)

variab_salud = ['asociaciones', 'casos', 'diagnósticos', 'dolores', 'experto', 'experta', 'expertas', 'hospitales', 'infección', 
                'luchas', 'mentales', 'mes', 'mujer', 'mundiales', 'niño', 'objetivos', 'pandemias', 'patologías', 
                'persona', 'profesional', 'recurso', 'sanitario', 'síntoma', 'tipos', 'trastorno', 'universidades']
salud_new_vocab = salud_new_vocab + variab_salud

salud_new_vocab.sort()
print(salud_new_vocab)

['asociaciones', 'asociación', 'atención', 'año', 'años', 'carlos', 'caso', 'casos', 'casos', 'covid', 'cáncer', 'diagnóstico', 'diagnósticos', 'dolor', 'dolores', 'día', 'días', 'ecuador', 'enfermedad', 'enfermedades', 'españa', 'española', 'evitar', 'experta', 'expertas', 'experto', 'expertos', 'explica', 'falta', 'forma', 'frenillo', 'gripe', 'hospital', 'hospitales', 'indica', 'infecciones', 'infección', 'lucha', 'luchas', 'malaria', 'mental', 'mentales', 'mes', 'meses', 'migraña', 'millones', 'mujer', 'mujeres', 'mundial', 'mundiales', 'niño', 'niños', 'objetivo', 'objetivos', 'paciente', 'pacientes', 'pandemia', 'pandemias', 'patología', 'patologías', 'país', 'países', 'persona', 'personas', 'piel', 'prevención', 'problema', 'problemas', 'profesional', 'profesionales', 'protección', 'recurso', 'recursos', 'salud', 'sanitario', 'sanitarios', 'sida', 'sociedad', 'solar', 'síntoma', 'síntomas', 'tipo', 'tipos', 'trastorno', 'trastornos', 'tratamiento', 'tratamientos', 'universidad',

### 2.6. Glosario final

In [14]:
#La lista "vocabulario" será el resultado de sumar las tres listas anteriores sin palabras repetidas
vocabulario = deportes_new_vocab
vocabulario = vocabulario + [token for token in politica_new_vocab if token not in vocabulario]
vocabulario = vocabulario + [token for token in salud_new_vocab if token not in vocabulario]
vocabulario.sort()

print(vocabulario)

['actividad', 'actividades', 'acuerdo', 'acuerdos', 'anna', 'asociaciones', 'asociación', 'atención', 'atlético', 'audiencia', 'audiencias', 'año', 'años', 'baloncesto', 'brexit', 'británica', 'británicas', 'británico', 'británicos', 'bruselas', 'carlos', 'casado', 'caso', 'casos', 'cgpj', 'china', 'comisiones', 'comisión', 'competiciones', 'competición', 'congreso', 'congresos', 'conjunto', 'conjuntos', 'consejo', 'consejos', 'covid', 'crisis', 'cuarto', 'cuartos', 'cáncer', 'deporte', 'deportes', 'deportiva', 'deportivas', 'deportivo', 'deportivos', 'diagnóstico', 'diagnósticos', 'diego', 'dolor', 'dolores', 'día', 'días', 'ecuador', 'empresa', 'empresas', 'encuentro', 'encuentros', 'enero', 'enfermedad', 'enfermedades', 'equipo', 'equipos', 'españa', 'español', 'española', 'españolas', 'españoles', 'europea', 'europeas', 'europeo', 'europeos', 'evitar', 'experta', 'expertas', 'experto', 'expertos', 'explica', 'falta', 'fernández', 'final', 'finales', 'forma', 'formaciones', 'formaci

## 3. Implementación del algoritmo de clasificación

In [15]:
#Conjunto de documentos de entrenamiento del clasificador
X_Train = deportes + politica + salud
Y_Train = [0] * len(deportes) + [1] * len(politica) + [2] * len(salud)

#Conjunto de documentos de test (prueba) del clasificador
filenames_test = glob.glob('Test/*')
X_Test = load_documents(filenames_test)    
Y_Test = check_original_labels(filenames_test)

#Se aleatorizan todos los documentos, tanto los de entrenamiento como los osmetidos a las posteriores pruebas
c = list(zip(X_Train, Y_Train))

random.shuffle(c)

X_Train, Y_Train = zip(*c)

d = list(zip(X_Test, Y_Test, filenames_test))

random.shuffle(d)

X_Test, Y_Test, filenames_test = zip(*d)

### 3.1. TF_IDF

In [16]:
#Se aplica la métrica tf-idf vista en clase
vectorizer = TfidfVectorizer(
                analyzer = 'word',
                tokenizer = tokenize,
                vocabulary = vocabulario)

X_Train_TF = vectorizer.fit_transform(X_Train)
X_Test_TF =  vectorizer.transform(X_Test)

### 3.2. KNN y resultados

In [17]:
n = 4
X_Train_TF = preprocessing.normalize(X_Train_TF)
X_Test_TF = preprocessing.normalize(X_Test_TF)

knn = KNeighborsClassifier(n_neighbors=n,weights='uniform', metric='euclidean').fit(X_Train_TF, Y_Train)

score_train = knn.score(X_Train_TF,Y_Train)
proba_test = knn.predict_proba(X_Test_TF)
score_test = knn.score(X_Test_TF,Y_Test)


temas = {0:'Deportes', 1:'Politica', 2:'Salud'}


t = PrettyTable(['Documento', 'Probabilidad (%)', 'Tema'])
for document_name, proba in zip(filenames_test,proba_test):
    t.add_row([document_name.replace("Test\\", ""), max(proba)*100, temas[np.argmax(proba)]])

print(t)
print('Train Accuracy:', score_train * 100, '%')
print('Test Accuracy:', score_test * 100, '%')

+----------------------+------------------+----------+
|      Documento       | Probabilidad (%) |   Tema   |
+----------------------+------------------+----------+
| 4-test-politica.txt  |      100.0       | Politica |
|  10-test-salud.txt   |      100.0       |  Salud   |
|  13-test-salud.txt   |      100.0       |  Salud   |
| 12-test-politica.txt |      100.0       | Politica |
| 3-test-politica.txt  |       75.0       | Politica |
|  14-test-salud.txt   |      100.0       |  Salud   |
| 7-test-deportes.txt  |       75.0       | Deportes |
| 9-test-politica.txt  |      100.0       | Politica |
| 14-test-deportes.txt |      100.0       | Deportes |
| 5-test-deportes.txt  |      100.0       | Deportes |
|   8-test-salud.txt   |       75.0       |  Salud   |
| 2-test-deportes.txt  |       75.0       | Deportes |
| 6-test-deportes.txt  |      100.0       | Deportes |
| 13-test-politica.txt |       75.0       | Politica |
| 8-test-politica.txt  |      100.0       | Politica |
|  11-test